In [75]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import geopandas as gpd
from shapely.geometry import Point


In [64]:
df =pd.read_csv('shooting-1982-2023.csv')

df.head()

,case,location,date,summary,fatalities,injured,total_victims,location.1,age_of_shooter,prior_signs_mental_health_issues,...,weapons_obtained_legally,where_obtained,weapon_type,weapon_details,race,gender,latitude,longitude,type,year
0,Nashville religious school shooting,"Nashville, TN",3-27-23,"Audrey Hale, 28, who was a former student at t...",6,1,6,School,28,-,...,-,-,"semiautomatic rifle, semiautomatic handgun",-,-,"F (identified as transgender, per police)",-,-,Mass,2023
1,Michigan State University shooting,"East Lansing, Michigan",2-13-23,"Anthony D. McRae, 43, opened fire at Berkey Ha...",3,5,8,School,43,-,...,yes,-,semiautomatic handguns,-,Black,M,-,-,Mass,2023
2,Half Moon Bay spree shooting,"Half Moon Bay, California",1-23-23,"Chunli Zhao, 67, suspected of carrying out the...",7,1,8,workplace,67,-,...,-,-,semiautomatic handgun,-,Asian,M,-,-,Spree,2023
3,LA dance studio mass shooting,"Monterey Park, California",1-21-23,"Huu Can Tran, 72, fled the scene in a white va...",11,10,21,Other,72,yes,...,-,-,semiautomatic assault weapon (Details pending),-,Asian,M,-,-,Mass,2023
4,Virginia Walmart shooting,"Chesapeake, Virginia",11-22-22,"Andre Bing, 31, who worked as a supervisor at ...",6,6,12,Workplace,31,-,...,-,-,semiautomatic handgun,-,Black,M,-,-,Mass,2022


In [65]:
df.shape

(141, 21)

In [66]:
df.isnull().sum()

case                                0
location                            0
date                                0
summary                             0
fatalities                          0
injured                             0
total_victims                       0
location.1                          0
age_of_shooter                      0
prior_signs_mental_health_issues    0
mental_health_details               0
weapons_obtained_legally            0
where_obtained                      0
weapon_type                         0
weapon_details                      1
race                                0
gender                              0
latitude                            0
longitude                           0
type                                0
year                                0
dtype: int64

In [67]:
df.dtypes

case                                object
location                            object
date                                object
summary                             object
fatalities                           int64
injured                             object
total_victims                       object
location.1                          object
age_of_shooter                      object
prior_signs_mental_health_issues    object
mental_health_details               object
weapons_obtained_legally            object
where_obtained                      object
weapon_type                         object
weapon_details                      object
race                                object
gender                              object
latitude                            object
longitude                           object
type                                object
year                                 int64
dtype: object

In [68]:
# Change location columns to float for mapping
df[['latitude','longitude']] = df[['latitude','longitude']].apply(pd.to_numeric, errors='coerce')

# Change date to datetime type
df['date'] = pd.to_datetime(df['date'], errors='coerce', format='%Y-%m-%d')

In [69]:
# Change these columns to int type

# Getting an invalid literal "fewer than 10" error when trying to convert the injured column to int, changing it to 0
# df['injured'].value_counts()
 
# Drop the row containing the specific value in any of the columns
df = df[~df[['injured', 'total_victims', 'age_of_shooter']].apply(lambda x: x.str.contains('TK - "fewer than 10"')).any(axis=1)]
df = df[~df[['injured', 'total_victims', 'age_of_shooter']].apply(lambda x: x.str.contains('-')).any(axis=1)]

# Convert columns to integers
df[['injured', 'total_victims', 'age_of_shooter']] = df[['injured', 'total_victims', 'age_of_shooter']].astype(int)

In [70]:
df.dtypes

case                                        object
location                                    object
date                                datetime64[ns]
summary                                     object
fatalities                                   int64
injured                                      int64
total_victims                                int64
location.1                                  object
age_of_shooter                               int64
prior_signs_mental_health_issues            object
mental_health_details                       object
weapons_obtained_legally                    object
where_obtained                              object
weapon_type                                 object
weapon_details                              object
race                                        object
gender                                      object
latitude                                   float64
longitude                                  float64
type                           

In [71]:
df.shape

(138, 21)

### Save the DataFrame to GEOJSON for Mapping in Leaflet

In [76]:
# Convert the DataFrame to a GeoDataFrame
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Save the GeoDataFrame as a GeoJSON file
gdf.to_file("data.geojson", driver='GeoJSON')

/opt/anaconda3/lib/python3.9/site-packages/geopandas/io/file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
